In [1]:
# You only need to execute this cell if you are using Google Golab.
# If you use GitHub Codespaces, everything is already installed.
!pip3 install tira ir-datasets python-terrier
!wget https://files.webis.de/software/pyterrier-plugins/custom-terrier-token-processing-1.0-SNAPSHOT-jar-with-dependencies.jar -O /root/.pyterrier/custom-terrier-token-processing-0.0.1.jar

--2024-06-19 10:57:25--  https://files.webis.de/software/pyterrier-plugins/custom-terrier-token-processing-1.0-SNAPSHOT-jar-with-dependencies.jar
Resolving files.webis.de (files.webis.de)... 141.54.132.200
Connecting to files.webis.de (files.webis.de)|141.54.132.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 499865236 (477M) [application/java-archive]
Saving to: ‘/root/.pyterrier/custom-terrier-token-processing-0.0.1.jar’

/root/.pyterrier/cu 100%[===================>] 476.71M  29.7MB/s    in 17s     

2024-06-19 10:57:44 (27.3 MB/s) - ‘/root/.pyterrier/custom-terrier-token-processing-0.0.1.jar’ saved [499865236/499865236]



In [2]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd
pd.set_option('display.max_colwidth', 0)

if not pt.started():
    pt.init(boot_packages=['mam10eks:custom-terrier-token-processing:0.0.1'])
    from jnius import autoclass


# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()


PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
#Index erstellen und ausführen des Batch
#Stanford Lemmatizer
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

#Build Index
indexer = pt.IterDictIndexer("/tmp/index2", overwrite=True, stemmer='StanfordLemmatizer')
index_ref = indexer.index(pt_dataset.get_corpus_iter())

#Load Index
index = pt.IndexFactory.of(index_ref)

bm25 = pt.BatchRetrieve(index, wmodel="BM25")

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  46%|████▋     | 59020/126958 [03:31<04:44, 239.19it/s]

In [ ]:
# #Localtest
# pt.Experiment(
#     [bm25], 
#     pt_dataset.get_topics(), 
#     pt_dataset.get_qrels(), 
#     eval_metrics=['P_1000', 'map', 'recip_rank']
#     # names=['BM25'],
#     # baseline=0
#     )


In [ ]:
print('First, we have a short look at the first three topics:')

pt_dataset.get_topics('text').head(3)

In [ ]:
print('Now we do the retrieval...')
run = bm25(pt_dataset.get_topics('text'))

print('Done. Here are the first 10 entries of the run')
run.head(10)

In [ ]:
#The output of a prototypical retrieval system is a run file. 
#This run file can later (optimally in a different notebook) be statistically evaluated.
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs')